<a href="https://colab.research.google.com/github/HammadN98/pytorch/blob/main/06_pytorch_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Pytorch Tranfer Learning

O que eh Transfer Learning?

Transfer Learning envolve pegar os parametros de um modelo que foi trainado em outros dataset e aplicar-los no nosso proprio ploblema

* Pretrained model = foundation models

In [ ]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)


2.4.1+cu121
0.19.1+cu121


In [ ]:
#COpiado da estrutura do curso, modificado apenas para manter a pasta "data" ao baixar o repositorio para o collab
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .

    #!mv pytorch-deep-learning/data .

    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4356, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 4356 (delta 213), reused 253 (delta 177), pack-reused 4035 (from 1)
Receiving objects: 100% (4356/4356), 654.51 MiB | 29.81 MiB/s, done.
Resolving deltas: 100% (2584/2584), done.
Updating files: 100% (248/248), done.


In [ ]:
#COnfigurando o dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# 1. Conseguindo os dados

Baixandoi os dados: Pizza, Steak, Sushi para construir um modelo com transfer learning

In [ ]:
import os
import zipfile
import requests

from pathlib import Path



#Configurando o caminho
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

#Se a pasta das imagens nao existe, baixar e prepara
if image_path.is_dir():
  print(f"Diretrio {image_path} existe, pulando download...")
else:
  print(f"O diretorio {image_path} nao foi encontrado, baixando...")
  image_path.mkdir(parents=True, exist_ok=True)

  #Baixando dados -> pizza, steak, sushi
  with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Baixando pizza, steak, sushi...")
    f.write(request.content)

  # Unzip pizza, steak, sushi
  with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
      print("Unzipping pizza, steak, sushi data...")
      zip_ref.extractall(image_path)

  # Remove .zip
  os.remove(data_path / "pizza_steak_sushi.zip")


O diretorio data/pizza_steak_sushi nao foi encontrado, baixando...
Baixando pizza, steak, sushi...
Unzipping pizza, steak, sushi data...


In [ ]:
#Configurando caminho dos diretorios
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

# 2. Criando Datasets e DataLoaders

Agora que se tem os dados, vamos precisar transformar em DataLoaders Pytorch, e para isso sera utilizado o `datas_setup.py` e a funcao `create_dataloaders()`, que foi feita no modulo passado.

Uma coisa para se pensar eh, como iremos **transformar** os dados?

Apartir da versao 0.13+ do torchvision eh possivel fazer de duas maneiras:

1. Criar os transforms de maneira manual - Aonde definimos por quais trnasforms queremos que os dados passem.
2. De maneira automatica: Com base no modelo que sera usado, ele definira os transforms.

**NOTE** Quando usado um modelo pre-treinado, eh importante que os dados(incluindo os custom proprios) que iremos usar sejam trnassformados do mesma menira que os dados em que o modelo foi originalmente treinado.

## 2.1 Crianado transforms com `torchvision.models` (Maneira manual)

`torchvision.models` contem modelos pre-treinados(ou seja modelos pronto para transfer learning) direto no torchvision.

>All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224.
>The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].
>
>

In [ ]:
from torchvision import transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
manual_transforms = transforms.Compose([
                                        transforms.Resize((224, 224)), #Tamanho para 224 224 (altura x largura)
                                        transforms.ToTensor(), # Transforma em range [0, 1]
                                        normalize]) #Garante que as imagnes tenham a mesma distribuicao do ImageNet(AOnde nosso modelo foi treinado)


In [ ]:
from going_modular.going_modular import data_setup
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms,
                                                                               batch_size=32)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7ca52067f310>,
 ['pizza', 'steak', 'sushi'])

## 2.2 Criando transforms com `torchvision.models` (Automatico)

In [ ]:
#PEgando um set do pesos do modelo pre treinado
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT #DEAFOULT = melhores pesos disponiveis
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [ ]:
#Conseguindo o transforms que foi usado para criar os pesos do modelo pre-treinado
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [ ]:
#Criando dataloaders usando o transforms automatico
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=auto_transforms,
                                                                               batch_size=32)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7ca5205ce590>,
 ['pizza', 'steak', 'sushi'])

# 3. Modelos Pre-Treinados

Lugares para conseguir modelos pre-treinados

1. PyTorch
2. Bibliotecas como a `timm` (torch image models)
3. HuggingFace Hub (PAra uma diversidade de modelos)
4. Paperswithcode (P modelo de tudo que eh tipo e cor)


## 3.1 Qual modelo pre-treinado usar?

Descobrir por meio da experimentacao

*Experiment, Experiment, Experiment*

A idea geral de usar **Transder Learning** que eh pegar um modelo ja validade para um problema similar ao nosso e customizar para o nosso problema.

Tres coisas a considerar:
1. Velocidade - O quao rapido roda?
2. Tamanho - O quao grande eh o modelo?
3. Performace - Quao bem os modelo se encaixa no problema a ser resolvido(E.G O quao bem o modelo classficia imagens? Para o FoodVision Mini)?

Levar em conta - AOnde o modelo vive(esta armazenado)?
  No dispositivo? Como um carreo autonomo? Ou em servidores?

O modelo escolhido para o nosso problema(Deploy do FoodVision Mini para dispositivos mobile), parece que a EffNetB0 eh uma das melhores opcoes em termos de Performace X Tamanho

Porem sempre vale lembrar, que se dinheiro nao for um problema pega o mais caro e azar.